In [1]:
import pandas as pd
import requests
from config import g_key
import numpy as np

In [2]:
#Read in CSV
file_name="LatLngSample.csv"
shark_df=pd.read_csv(file_name)
shark_df.head()

,Date,Year,Type,Country,ISO Code,Area,Location,Activity,Name,Sex,Age,Injury,Fatal (Y/N),Time,Species,Investigator or Source
0,9-Sep-21,2021,Unprovoked,USA,US,Florida,"Ponce Inlet, Volusia County",Surfing,Doyle Neilsen,M,!6,Minor injury to right arm,N,13h20,NaN,"Daytona Beach News-Journal, 9/14/2021"
1,5-Sep-21,2021,Unprovoked,AUSTRALIA,AU,New South Wales,Emerald Beach,Surfing,Timothy Thompson,M,31,FATAL,Y,10h30,White xhark,"B. Myatt, GSAF"
2,3-Sep-21,2021,Unprovoked,British Overseas Territory,TC,Turks and Caicos,NaN,NaN,male,M,NaN,Wrist bitten,N,NaN,NaN,Anonymous
3,28-Aug-21,2021,Unprovoked,USA,US,Texas,"Galveston Island, Galveston County",Boogie boarding,male,M,!!,Lacerations both sides of lower leg immediatel...,N,11h45,NaN,"T. Craig, GSAF & K. McMurray, TrackingShark.com"
4,3-Sep-21,2021,Unprovoked,USA,US,Louisiana,off Grand Isle,"Fishing, fell in water",male,M,64,Severe injury to leg,N,14h00,NaN,"S. Wagemann, GSAF"


In [3]:
shark_df=shark_df.fillna(value=" ")
shark_df

,Date,Year,Type,Country,ISO Code,Area,Location,Activity,Name,Sex,Age,Injury,Fatal (Y/N),Time,Species,Investigator or Source
0,9-Sep-21,2021,Unprovoked,USA,US,Florida,"Ponce Inlet, Volusia County",Surfing,Doyle Neilsen,M,!6,Minor injury to right arm,N,13h20,,"Daytona Beach News-Journal, 9/14/2021"
1,5-Sep-21,2021,Unprovoked,AUSTRALIA,AU,New South Wales,Emerald Beach,Surfing,Timothy Thompson,M,31,FATAL,Y,10h30,White xhark,"B. Myatt, GSAF"
2,3-Sep-21,2021,Unprovoked,British Overseas Territory,TC,Turks and Caicos,,,male,M,,Wrist bitten,N,,,Anonymous
3,28-Aug-21,2021,Unprovoked,USA,US,Texas,"Galveston Island, Galveston County",Boogie boarding,male,M,!!,Lacerations both sides of lower leg immediatel...,N,11h45,,"T. Craig, GSAF & K. McMurray, TrackingShark.com"
4,3-Sep-21,2021,Unprovoked,USA,US,Louisiana,off Grand Isle,"Fishing, fell in water",male,M,64,Severe injury to leg,N,14h00,,"S. Wagemann, GSAF"
5,21-Aug-21,2021,Unprovoked,USA,US,Florida,"Walton Beach Psrk, St. Lucie County",Surfing,Jeremy Carr,M,41,Left ankle and foot bitten,N,06h40,Bull shark,
6,13-Aug-21,2021,Unprovoked,CANADA,CA,Nova Scotia,Cape Breton Island,Swimming,Taylor Boudreau-Deveaux,F,21,Thigh injured,N,`17h00,,"J. Marchand, GSAF"
7,7-Aug-21,2021,Provoked,AUSTRALIA,AU,Queensland,North Stradbroke Island,Harassing sharks,Rick Manning,M,,"No Injury, jet ski bitten PROVOKED INCIDENT",N,,,"J. Marchand and S. De Marchi, GSAF"
8,7-Aug-21,2021,Unprovoked,USA,US,Florida,"New Smyrna Beach, Volusia County",Surfing,male,M,35,Minor cuts to right foot and ankle,N,10h00,,"J. Marchand, GSAF"
9,7-Aug-21,2021,Unprovoked,USA,US,Florida,"Key Largo, Monroe County",Snorkeling,Lucas Cruz,M,15,Lacerations to legs,N,07h51,,"R. B, GSAF"


In [4]:
iso=shark_df["ISO Code"].tolist()
loc=shark_df["Location"].tolist()
iso_loc_zip=zip(iso, loc)
iso_loc=list(iso_loc_zip)


In [5]:
lat_lng_list=[]

print("Beginning API")
print("---------")

#Create counters
record_count= 1
set_count=1

for iso, loc in enumerate(iso_loc):     
    ISO=iso_loc[iso][0]
    LOC=iso_loc[iso][1].replace(" ", "+")
    url="https://maps.googleapis.com/maps/api/geocode/json?address=" + LOC + "+District&components=country:" + ISO + "&key=" + g_key

    try:
        #Parse JSON data
        lat_lng=requests.get(url).json()
        #Needed data
        city_lat=lat_lng["results"][0]["geometry"]["location"]["lat"]
        city_lng=lat_lng["results"][0]["geometry"]["location"]["lng"]
        lat_lng_list.append({"Lat":city_lat,
                            "Lng" : city_lng})
        
    except:
        print ("Lat/Long Not Found")
        lat_lng_list.append({"Lat":0,
                            "Lng" :0})
        pass

print ("--------")
print ("API Run Complete")

Beginning API
---------
--------
API Run Complete


In [ ]:
lat_lng_df=pd.DataFrame(lat_lng_list)
lat_lng_df

In [ ]:
shark_df["Lat"] = lat_lng_df["Lat"]
shark_df["Lng"]= lat_lng_df["Lng"]
shark_df

In [ ]:
#Export to new CSV
shark_df.to_csv("shark_data_with_lat_lng.csv")